In [7]:
import acquire as a

In [2]:
mylist = [
'google/exposure-notifications-android', 
'MohGovIL/hamagen-react-native', 
'phildini/stayinghomeclub', 
'rbignon/doctoshotgun',
'OxCGRT/covid-policy-tracker',
'covidpass-org/covidpass',
'AaronWard/covidify',
'MoH-Malaysia/covid19-public',
'GuangchuangYu/nCov2019',
'wcota/covid19br',
'covidatlas/coronadatascraper',
'reichlab/covid19-forecast-hub',
'swsoyee/2019-ncov-japan',
'CITF-Malaysia/citf-public',
'Ank-Cha/Social-Distancing-Analyser-COVID-19',
'VinAIResearch/BERTweet',
'amodm/api-covid19-in',
'JohnCoene/coronavirus',
'GoogleCloudPlatform/covid-19-open-data',
'deepset-ai/COVID-QA',
'bhattbhavesh91/cowin-vaccination-slot-availability',
'ccodwg/Covid19Canada',
'HzFu/COVID19_imaging_AI_paper_list',
'aatishb/covidtrends',
'github/covid-19-repo-data',
'anshumanpattnaik/covid19-full-stack-application',
'open-covid-19/data',
'paulvangentcom/python_corona_simulation',
'fluttercandies/ncov_2019',
'trekhleb/covid-19',
'COVID-19-electronic-health-system/Corona-tracker',
'wobsoriano/covid3d',
'dsfsi/covid19za'
]

In [3]:
#process only one repo
test= a.process_repo('dsfsi/covid19za')

In [4]:
#take a look
test

{'repo': 'dsfsi/covid19za',
 'language': 'Jupyter Notebook',
 'readme_contents': "# Coronavirus COVID-19 (2019-nCoV) Data Repository for South Africa\n\n[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.3819126.svg)](https://doi.org/10.5281/zenodo.3819126) [![dsJournal](https://img.shields.io/badge/DSJournal-10.5334-B31B1B.svg)](https://doi.org/10.5334/dsj-2020-019)\n\nCoronavirus COVID-19 (2019-nCoV) Data Repository for South Africa created, maintained and hosted by [Data Science for Social Impact research group](https://dsfsi.github.io/), led by Dr. Vukosi Marivate, at the University of Pretoria.\n\n**Disclaimer:** We have worked to keep the data as accurate as possible. We collate the COVID 19 reporting data from NICD and DoH. We only update that data once there is an official report or statement. For the other data, we work to keep the data as accurate as possible. If you find errors. Make a pull request.\n\n*If you use this repo for any research/development/innovation, please co

In [8]:
scrape_data = a.scrape_github_data()

MissingSchema: Invalid URL '': No schema supplied. Perhaps you meant http://?

In [ ]:
df = pd.DataFrame(scrape_data)
df.info()

# Prepare

In [ ]:
df = df.rename({'readme_contents':'original'},axis=1)
df['clean'] = df.original.apply(basic_clean)
df['lemmatized'] = df.clean.apply(lemmatize)
df['stopwords_removed'] = df.lemmatized.apply(remove_stopwords)
df.head()